In [1]:
!rm -rf Eunomia
!git clone https://github.com/ouaguenouni/Eunomia
!pip install -r Eunomia/requirements.txt

Cloning into 'Eunomia'...
remote: Enumerating objects: 532, done.
remote: Counting objects: 100% (532/532), done.
remote: Compressing objects: 100% (204/204), done.
remote: Total 532 (delta 396), reused 458 (delta 322), pack-reused 0
Receiving objects: 100% (532/532), 3.49 MiB | 2.26 MiB/s, done.
Resolving deltas: 100% (396/396), done.


In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from Eunomia.preferences import *
from Eunomia.additive_functions import *
from Eunomia.alternatives import *
from Eunomia.sampling import *
from Eunomia.mcmc import *
from Eunomia.degree import *
from Eunomia.experiments import *


In [ ]:
import time

TEST_NAME = "SX_IJCAI_N"
REPLACE = False
sigma_weights = 1
n = 5
sigma_w = 1
sigma_p = 1
n_samples = 500
k=2
for n in np.arange(8,15):
    m = 2**(n-2)
    for iter_ in range(10):
        run_d = {
            "iter_":int(iter_),
            "n": int(n),
            "k": int(k),
            "m": int(m),
            "sigma_w": float(sigma_w),
            "sigma_p": float(sigma_p),
            "n_samples": n_samples
        }
        found = find_experiment_file(run_d, TEST_NAME)
        if found:
            date = found
            print(f"Found file: {file_name}")
            if REPLACE:
                print('Replacing it...')
            else:
                continue
        else:
            file_name = compute_experiment_file_name(run_d, TEST_NAME)
            print("Registering the results in: ", file_name)
        ####
        theta = generate_additive_theta(n, k)
        weights = generate_normal_weights(theta, sigma_weights)
        alternatives = generate_random_alternatives_matrix(m, n)
        ranks = compute_ws_ranks(alternatives, theta, weights)
        t_sv = compute_semivalues(n, theta, weights, lambda x:1)
        preferences = PreferenceModel(alternatives, ranks)
        data = preferences.generate_preference_matrix(theta)
        data = torch.tensor(data).float()
        t = time.time()
        model = posterior_sampling_model(data, sigma_w=sigma_w, sigma_p=sigma_p)
        diag, sampled_weights, sigmas = sample_model(model, data, "w", "sigma", warmup_steps=500, num_samples=n_samples, return_diag=True)
        t = time.time() - t
        accs_d = get_acc_distribution(data, sampled_weights, sigmas)
        predicted_rankings = [np.argsort(compute_semivalues(n, theta, weights, lambda x:1))[::-1] for weights in sampled_weights]
        kt_d = get_kt_distribution(predicted_rankings, np.argsort(t_sv))
        ###
        
        file_path = record_experiment_results(run_d, TEST_NAME)
        
        run_d["time"] = t
        run_d["weights"] = weights.tolist()
        run_d["predicted_rankings"] = [i.tolist() for i in predicted_rankings]
        run_d["accuracy_distribution"] = [i.tolist() for i in accs_d]
        run_d["kt_d"] = [i.tolist() for i in kt_d]
        run_d["acceptance_rate"] = diag["acceptance rate"]["chain 0"]
        run_d["w_eff"] = [float(i) for i in diag["w"]["n_eff"].numpy()]
        run_d["w_rhat"] = [float(i) for i in diag["w"]["r_hat"].numpy()]
        
        print("diag: ", diag)
        with open(file_path, 'w') as file:
            yaml.dump(run_d, file, default_flow_style=False)


Registering the results in:  SX_IJCAI_N/2024-01-10/iter_-0_n-8_k-2_m-64_sigma_w-1.0_sigma_p-1.0_n_samples-500.yaml


Sample: 100%|████████████████████████████| 1000/1000 [06:33,  2.54it/s, step size=1.54e-02, acc. prob=0.705]


diag:  {'sigma': OrderedDict([('n_eff', tensor(81.4990)), ('r_hat', tensor(1.0275))]), 'w': OrderedDict([('n_eff', tensor([ 54.6111,  79.1406, 217.8656,  97.2304,  76.8923,  77.4986,  48.8967,
        153.3358,  76.9271,  51.5287,  44.4582,  51.2885,  54.8569,  59.9436,
         44.0988,  42.0962,  54.7177, 114.7301,  71.3614,  45.2901,  74.6922,
        181.3629,  52.1294,  84.0421,  45.3792,  41.6921,  63.2260, 125.3802,
        220.7271,  60.7923, 128.6418, 132.6886,  55.7039,  55.1778,  54.0603,
         48.0959])), ('r_hat', tensor([1.0123, 1.0003, 1.0052, 1.0056, 1.0061, 1.0087, 1.0230, 0.9980, 0.9984,
        1.0101, 1.0264, 1.0109, 1.0129, 1.0231, 1.0237, 1.0243, 1.0230, 0.9980,
        1.0079, 1.0116, 1.0029, 0.9989, 1.0136, 1.0004, 1.0204, 1.0269, 1.0080,
        1.0015, 1.0057, 1.0314, 1.0126, 1.0001, 1.0366, 1.0286, 1.0118, 1.0121]))]), 'divergences': {'chain 0': [41, 67, 87, 118, 119, 124, 175, 186, 188, 194, 209, 212, 232, 233, 236, 241, 246, 266, 274, 276, 277, 278, 279,

Sample: 100%|████████████████████████████| 1000/1000 [07:03,  2.36it/s, step size=1.73e-02, acc. prob=0.876]


diag:  {'sigma': OrderedDict([('n_eff', tensor(153.5775)), ('r_hat', tensor(1.0003))]), 'w': OrderedDict([('n_eff', tensor([165.6020,  99.1497, 107.7830,  95.8946, 148.2458, 172.2292, 238.5741,
        271.9439, 206.7361,  93.4071, 216.7930, 104.7597, 104.4216, 252.0884,
        101.0451, 208.3377, 261.7551, 117.7100, 214.3439, 163.2884, 145.3738,
        200.5380, 113.4986, 104.4005, 129.4939, 215.7637, 209.5300, 104.3457,
        149.3185, 129.6595, 102.9055, 116.5892, 120.1667,  93.0739, 234.2477,
        243.1942])), ('r_hat', tensor([0.9981, 1.0313, 1.0212, 1.0359, 0.9990, 1.0008, 0.9980, 0.9992, 1.0015,
        1.0108, 0.9998, 1.0179, 1.0056, 1.0088, 1.0116, 1.0075, 1.0116, 1.0032,
        1.0141, 1.0136, 1.0056, 0.9989, 1.0006, 1.0159, 0.9985, 1.0053, 1.0199,
        1.0037, 0.9998, 1.0083, 1.0078, 1.0139, 1.0148, 1.0149, 0.9980, 0.9991]))]), 'divergences': {'chain 0': [2, 27, 33, 37, 120, 130, 231, 270, 295, 370, 391, 396, 415]}, 'acceptance rate': {'chain 0': 1.0}}
Registering

...

Sample: 100%|████████████████████████████| 1000/1000 [32:12,  1.93s/it, step size=5.98e-03, acc. prob=0.630]


diag:  {'sigma': OrderedDict([('n_eff', tensor(16.7517)), ('r_hat', tensor(1.0707))]), 'w': OrderedDict([('n_eff', tensor([ 4.6308,  4.6994,  4.6885,  4.6936,  4.6666,  4.9496,  4.6783,  4.6161,
         4.7508,  4.6617,  4.7354,  4.7107,  4.6919,  4.7752, 11.5502,  4.6815,
         4.7140,  4.7249, 15.2681,  4.6359,  4.6679,  4.6964,  4.6543,  4.6836,
         4.6686, 12.0499,  4.7369,  4.6816,  4.6871,  4.6770,  4.6990,  4.7243,
         4.6772,  4.6677,  4.6827,  4.7776,  4.6924,  4.7352,  4.6788,  4.6883,
         4.7228,  4.7399, 45.9986,  4.6692,  7.5144,  4.6687,  4.7165,  4.6772,
         4.7110,  4.7868,  4.6583,  4.6748,  4.6843,  4.7159,  4.7913,  4.6458,
         4.7118,  4.7042,  4.7406,  4.7192,  4.6768,  4.7405,  4.6746,  4.8237,
         4.6716,  4.8312])), ('r_hat', tensor([1.3859, 1.3685, 1.3691, 1.3786, 1.3755, 1.3898, 1.3823, 1.3901, 1.3580,
        1.3866, 1.4077, 1.3677, 1.3659, 1.3531, 1.0435, 1.3712, 1.3656, 1.3616,
        1.1189, 1.3855, 1.3703, 1.3646, 1.3747

Sample: 100%|████████████████████████████| 1000/1000 [45:20,  2.72s/it, step size=6.16e-03, acc. prob=0.739]


diag:  {'sigma': OrderedDict([('n_eff', tensor(68.6609)), ('r_hat', tensor(0.9980))]), 'w': OrderedDict([('n_eff', tensor([ 6.1038,  5.5754,  5.5646, 18.3948,  5.6856,  5.5641,  5.5978,  6.1490,
         5.5991,  5.5596,  5.5699,  5.5519,  5.5493,  5.5410,  5.5491,  5.5634,
         5.5532,  5.5543,  5.5333,  5.5907,  6.0475,  5.5712,  6.2903,  5.5589,
         5.5535,  5.6526,  7.3935,  5.5719,  5.5529,  5.5331,  5.5553,  5.5769,
         5.7321,  5.6298,  5.5459,  5.5600,  5.6271,  5.5755,  5.5682,  5.5561,
         5.9583,  5.5951,  5.5738,  5.5469,  5.5679,  5.5413,  5.5499,  5.5684,
        10.9501,  5.5588,  5.5989,  5.5718,  5.5645,  5.5587,  5.5374,  5.5696,
         5.6303,  5.5609,  5.5521,  5.5898,  5.5384,  5.5604,  5.5948,  5.6162,
         5.5959,  5.5752])), ('r_hat', tensor([1.1450, 1.1609, 1.1619, 1.0621, 1.1555, 1.1626, 1.1555, 1.1496, 1.1634,
        1.1615, 1.1607, 1.1620, 1.1633, 1.1621, 1.1631, 1.1635, 1.1614, 1.1609,
        1.1625, 1.1594, 1.1371, 1.1606, 1.1445

Warmup:  34%|█████████▊                   | 340/1000 [16:29,  5.13s/it, step size=1.94e-03, acc. prob=0.775]